# Item-Item Top-N Recommendations

In this excercise we will implement a simple top-N recommender, evaluate the algorithms, and then call algorithms from the Surprise package. In top-N recommendations the algorithm is requested to produce a list of n items that the user will be interested in. 
In this particular execercise we will work with the escape room dataset that we have discussed in class.

First, let's load the dataset, which is already split by time into a training set and a test set:

In [3]:
import pandas as pd
train_set_path = 'resources//train_numerized_with_anon.csv'
test_set_path = 'resources//test_numerized_with_anon.csv'

train_set = pd.read_csv(train_set_path, parse_dates=[3], index_col='index')
test_set = pd.read_csv(test_set_path, parse_dates=[3], index_col='index')

users_in_train = train_set.userID.unique()
test_set = test_set[test_set.userID.isin(users_in_train)]

We can take a look at the structure of the dataset:

In [4]:
train_set.head()

,userID,itemID,rating,timestamp
index,,,,
0,0,0,10,11/11/2015
1,1,0,10,11/11/2015
2,2,0,8,11/11/2015
3,3,0,10,11/11/2015
4,4,0,10,11/11/2015


## Part 1: Recommend Most Popular Items 

Now we can begin implementing the our first algorithm, that recommends to the user the list of most popular items. Although this is not a personalized approach, in many cases, this is not a bad idea - popular items are popular because everybody choose them, so there is a high likelihood that recommended popular items will be indeed chosen by the user.

In the code below, fill in the missing parts. The algorithm has a training method, where item popularity is computed, and a recommendation method, where the list of popular items.

In [17]:
class MostPopular:

    def __init__(self):
        self.item_ratings_sorted = None
        self.train_set = None

    def learn_model(self, train_set):
        self.train_set = train_set
        #1) Add code to set the item_ratings_sorted to the list of items in the training set, 
        #ordered by decreasing popularity (i.e., the number of users who have chosen an item)
        self.item_ratings_sorted = train_set['itemID'].value_counts()
        

    def get_top_n_recommendations(self, test_set, top_n):
        result = {}
        already_ranked_items_by_user = self.train_set.groupby('userID')['itemID'].apply(list)
        top_list_item = self.item_ratings_sorted.keys().tolist()
        #For each user in the test set compute recommendations
        for userID in test_set.userID.unique():
            result[str(userID)] = [i for i in top_list_item if i not in already_ranked_items_by_user[userID]][:top_n]
            
            #2) Add to the result the top N items from the popular list
            #3) Remove items that the user has already chosen in the training set (already_ranked_items_by_user)
        return result

    def clone(self):
        pass


Now we can call the most popular algorithm to deliver a list of reocmmendations. The code below prints the list of top 5 recommended items for user with ID 431.

In [20]:
popular = MostPopular()
popular.learn_model(train_set)
popular_recs = popular.get_top_n_recommendations(test_set,top_n=5)
print(popular_recs['431'])
assert popular_recs['431'] == [53, 26, 68, 85, 16], 'Wrong computation of popular items'

[53, 26, 68, 85, 16]


## Part 2 - Item-Item Recommendations

We now learn a slightly more sophisticated model, that uses item-item similarities. Given such a similarity score, we can recommend to a user items that are most similar to the items that the user has chosen in the past. One such useful similarity metric is the Jaccard coefficient. For two items i1 and i2, the Jaccard similarity is the number of users who have chosen both i1 and i2, divided by the number of users who have chosen either i1 or i2. That is, given the list of users who have chosen i1 and the list of users who have chosen i2, the Jaccard similarity is the intersection of the lists, divided by the union of the lists.

In practice, to expedite the recommendation process, and hence reduce online latency, we will compute the item-item co-occurence matrix in the model learning phase. Then, online, when recommendations are requested, we only need to compute for each item that the user has already chosen in the past, the Jaccard scores for the other items.

As the user has chosen several items in the past, we need to aggregate the Jaccard scores. That is, if the user has previously chosen i1 and i2, item i3 has two scores J(i1,i3) and J(i2,i3), and an aggregation of the scores is needed. There are two popular aggregation functions - sum and max. Empirically, max typically has better perfromance.

Fill in the missing parts in the code below.

In [59]:
from tqdm import tqdm
import numpy as np
import operator
from collections import defaultdict

class Jaccard:

    def __init__(self):
        self.item_ratings_sorted = None
        self.train_set = None
        self.item_item_counts = dict()
        self.item_counts = None

    def learn_model(self, train_set):
        print('Started training')
        self.train_set = train_set
        self.item_counts =  self.train_set.groupby('itemID')['userID'].agg('count')
        
        maxpair = 0
        pbar = tqdm(total=len(train_set.userID.unique()))
        
        #Iterating over all item pairs is inefficient. It is better to iterate only over pairs of items that were chosen together.
        #Instead, we will iterate over the users, and for each user, and each two items that the user has chosen, increase the count
        for u in train_set.userID.unique():
            pbar.update(1)
            userData = self.train_set[self.train_set.userID == u]
            
            #4) For each pair of items in the user data - increase the counts in self.item_item_counts
            for item1 in userData['itemID']:
                for item2 in userData['itemID']:
                    pair_count = self.item_item_counts.get((item1,item2), 0) + 1
                    self.item_item_counts[(item1,item2)] = pair_count

        pbar.close()
        print('Done training')
            
            
            

    def get_top_n_recommendations(self, test_set, top_n):
        print('Started computing recommendations')
        result = {}
        already_ranked_items_by_users = self.train_set.groupby('userID')['itemID'].apply(list)
        users_by_item = self.train_set.groupby('itemID')['userID'].apply(list)
        all_items = self.train_set['itemID'].unique()
        pbar = tqdm(total=len(test_set.userID.unique()))
        
        #For each user in the test set compute recommendations
        for userID in test_set.userID.unique():
            pbar.update(1)
            result[str(userID)] = []
            #maxvalues will maintain for each potential item to recommend its highest Jaccard score.
            maxvalues = defaultdict(int)
            
            #4) Iterate over the items that the user has already chosen.
            #For each such item compute its Jaccard correlation to other items based on the item_item_counts.
            #Aggregate into maxvalues using max.
            #Avoid recommending items that the user has already chosen in the training set.
            ranked_by_user = already_ranked_items_by_users[userID]
            not_ranked_items_by_users = np.setdiff1d(all_items, ranked_by_user, assume_unique=True)
            for item1 in ranked_by_user:
                for item_not_ranked in not_ranked_items_by_users:
                    nb_count = self.item_item_counts.get((item1, item_not_ranked), 0)
                    union_count = len(np.unique(users_by_item.iloc[item1].append(users_by_item.iloc[item_not_ranked])))
                    jaccard_score = nb_count / union_count
                    if maxvalues[item_not_ranked] < jaccard_score:
                        maxvalues[item_not_ranked] = jaccard_score
            
            #Now we just take the top N items by decreasing Jaccard
            top_list = sorted(maxvalues.items(), key=lambda kv : -kv[1])
            i=0
            j=0
            while i < top_n and j < len(top_list):
                itemID = top_list[j][0]
                j = j + 1
                result[str(userID)].append(itemID)
                i = i + 1
            
        pbar.close()
        print('Done computing recommendations')
        return result

    
    def clone(self):
        pass


The code below trains a Jaccard model and generates recommendations. Training will take a while, as we need to iterate over all users, and for each user go over her items in quadratic time.

In [60]:
jaccard = Jaccard()
jaccard.learn_model(train_set)
jaccard_recs = jaccard.get_top_n_recommendations(test_set,top_n=5)









  0%|          | 0/20197 [00:00<?, ?it/s]






  1%|          | 104/20197 [00:00<00:19, 1026.68it/s]

Started training









  1%|          | 203/20197 [00:00<00:19, 1013.81it/s]






  2%|▏         | 316/20197 [00:00<00:19, 1044.04it/s]






  2%|▏         | 432/20197 [00:00<00:18, 1073.02it/s]






  3%|▎         | 516/20197 [00:00<00:24, 815.03it/s] 






  3%|▎         | 607/20197 [00:00<00:24, 797.23it/s]






  4%|▎         | 709/20197 [00:00<00:22, 852.33it/s]






  4%|▍         | 797/20197 [00:00<00:22, 860.23it/s]






  4%|▍         | 893/20197 [00:00<00:21, 886.09it/s]






  5%|▍         | 981/20197 [00:01<00:22, 866.51it/s]






  5%|▌         | 1080/20197 [00:01<00:21, 899.06it/s]






  6%|▌         | 1170/20197 [00:01<00:22, 856.52it/s]






  6%|▋         | 1271/20197 [00:01<00:21, 895.84it/s]






  7%|▋         | 1375/20197 [00:01<00:20, 933.67it/s]






  7%|▋         | 1474/20197 [00:01<00:19, 948.86it/s]






  8%|▊         | 1590/20197 [00:01<00:18, 1003.51it/s]






  8%|▊         | 1692/20197 [00:01<00:18, 1005.03it/s]






  9%|▉         | 1794/20197 [00:01<0

 58%|█████▊    | 11779/20197 [00:10<00:06, 1235.88it/s]






 59%|█████▉    | 11904/20197 [00:10<00:06, 1239.27it/s]






 60%|█████▉    | 12030/20197 [00:10<00:06, 1243.69it/s]






 60%|██████    | 12155/20197 [00:10<00:06, 1233.24it/s]






 61%|██████    | 12280/20197 [00:11<00:06, 1238.12it/s]






 61%|██████▏   | 12405/20197 [00:11<00:06, 1241.23it/s]






 62%|██████▏   | 12530/20197 [00:11<00:06, 1233.48it/s]






 63%|██████▎   | 12657/20197 [00:11<00:06, 1242.60it/s]






 63%|██████▎   | 12782/20197 [00:11<00:05, 1236.57it/s]






 64%|██████▍   | 12906/20197 [00:11<00:06, 1208.94it/s]






 65%|██████▍   | 13028/20197 [00:11<00:06, 1179.89it/s]






 65%|██████▌   | 13147/20197 [00:11<00:05, 1176.62it/s]






 66%|██████▌   | 13270/20197 [00:11<00:05, 1191.31it/s]






 66%|██████▋   | 13392/20197 [00:12<00:05, 1197.51it/s]






 67%|██████▋   | 13512/20197 [00:12<00:05, 1183.56it/s]






 67%|██████▋   | 13631/20197 [00:12<00:05, 1178.35it/s]






 68%|███

Done training
Started computing recommendations









  0%|          | 0/227 [00:00<?, ?it/s]






  1%|          | 2/227 [00:00<01:23,  2.71it/s]






  1%|▏         | 3/227 [00:01<01:45,  2.12it/s]






  2%|▏         | 4/227 [00:01<01:21,  2.75it/s]






  3%|▎         | 6/227 [00:01<01:00,  3.64it/s]






  3%|▎         | 7/227 [00:02<01:31,  2.39it/s]






  4%|▎         | 8/227 [00:02<01:20,  2.73it/s]






  4%|▍         | 9/227 [00:03<01:22,  2.64it/s]






  5%|▍         | 11/227 [00:03<01:00,  3.54it/s]






  6%|▌         | 14/227 [00:03<00:45,  4.68it/s]






  7%|▋         | 16/227 [00:03<00:36,  5.71it/s]






  8%|▊         | 18/227 [00:03<00:34,  6.07it/s]






  9%|▉         | 20/227 [00:04<00:34,  6.03it/s]






  9%|▉         | 21/227 [00:04<00:46,  4.44it/s]






 10%|▉         | 22/227 [00:05<01:38,  2.08it/s]






 10%|█         | 23/227 [00:06<02:21,  1.44it/s]






 11%|█         | 24/227 [00:06<01:45,  1.93it/s]






 11%|█         | 25/227 [00:08<02:23,  1.41it/s]






 12%|█▏        | 28

Done computing recommendations


As a side note - as computing the ite-item counts takes a while (especially with Python), we are using here the progress bar from the tqdm package (https://pypi.org/project/tqdm/). You need to install tqdm, or remove the progress bar, which would of course is not needed for the algorithm to run.

## Part 3 - Comparing the Algorithms 

We now want to compare the recommendation lists to see which one is better. In top-N recommendations it is popular to computer the Precision@N metric - the portion of recommended items that were chosen by users in the test set. This is typically a reasonable metric for real systems, where one wants to optimize the number of recommended items that are chosen.

We compute Precision@N by comparing the number of recommendations chosen by the users, divided by the number of overall recommendations.

Fill in the missing parts in the code below:

In [75]:
test_set[test_set.userID == 431]

,userID,itemID,rating,timestamp
index,,,,
69,431,366,9,14/09/2018
70,431,342,6,14/09/2018
71,431,362,7,14/09/2018
72,431,365,8,14/09/2018
933,431,85,7,14/10/2018
934,431,372,8,14/10/2018


In [111]:
def compute_precision(test_set, recommendations):
    #hits is the number of items that were recommended and chosen
    hits = 0
    #recs is the total number of recommended items
    recs = 0
    
    for u in test_set.userID.unique():
        userData = test_set[test_set.userID == u]
        userRecs = recommendations.get(str(u))
        #5) Compute here the number of hits. Update hits and recs accordingly.
        userItems = userData.itemID.tolist()
#         userItems = np.sort(userData.itemID)[:5].tolist()
        hits +=  len(np.intersect1d(userItems, userRecs))
        recs +=  len(userRecs)
    return hits / recs
        
    

In [112]:
p1 = compute_precision(test_set, jaccard_recs)
p2 = compute_precision(test_set, popular_recs)
print("Jaccard=", p1, "  Popularity=", p2)

Jaccard= 0.03259911894273128   Popularity= 0.027312775330396475


The precision values for this dataset may seem pretty low, but this is typical for many top-N problems. It is important not to compute metrics that hide the true values, such as AUC, but to acknowledge the perfromance of the system in the application.

## Part 4 - Calling Algorithms from the Surprise Package

There are many existing recommendation algorithms available. We will now see how we can call algorithms from the Surprise package. 

The code below adds a wrapper around the algorithm to transform the resulting recommendations into our desired format.

#### NOTE: 
To run the code below you first have to install surprise (http://surpriselib.com/). surprise requires scipy >=1.0, so update if needed.

To install: pip install scikit-surprise or, if you're using anaconda:  conda install -c conda-forge scikit-surprise).  

In [65]:
import sys, string, os
import pandas as pd
import itertools
from tqdm import tqdm
import numpy as np
import operator
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import PredefinedKFold
from surprise.prediction_algorithms import *



class SurpriseRecMethod():

    #method will be the specific Surprise algorithm that we will call
    def __init__(self, method):
        self.method = method

    def fit(self, train_set):
        self.train_set = train_set


    def get_top_n_recommendations(self, test_set, top_n):
        self.test_set = test_set

        #Surprise requires a slightly different input data format, so we use two different CSVs
        test_path_tmp = "resources//test_file.csv"
        train_path_tmp = "resources//train_file.csv"

        self.train_set.to_csv(train_path_tmp, index=False, header=False)
        self.test_set.to_csv(test_path_tmp, index=False, header=False)

        fold_files = [(train_path_tmp, test_path_tmp)]
        reader = Reader(rating_scale=(1, 10), line_format='user item rating', sep=',')
        data = Dataset.load_from_folds(fold_files, reader=reader)

        for trainset, testset in PredefinedKFold().split(data):
            self.method.fit(trainset)

        already_ranked_items_by_users = self.train_set.groupby('userID')['itemID'].apply(list)

        recommendations = {}
        pbar = tqdm(total=len(self.test_set.userID.unique()))
        for userID in self.test_set.userID.unique():
            pbar.update(1)

            if userID not in self.train_set.userID.unique():
                recommendations[str(userID)] = []
                continue

            items_expected_ranking = {}
            for itemID in self.train_set.itemID.unique():
                if itemID in already_ranked_items_by_users[userID]:
                    continue
                #We call here the specific Surprise method that we use for this model
                #The method predicts a score for a given item
                predicted = self.method.predict(str(userID), str(itemID), clip=False)
                items_expected_ranking[itemID] = predicted[3]
                
            #Now we just sort by decreasing scores and take the top N
            sorted_predictions = sorted(items_expected_ranking.items(), key=operator.itemgetter(1))
            sorted_predictions.reverse()
            sorted_predictions = [x[0] for x in sorted_predictions]
            user_recommendations = sorted_predictions[:top_n]
            recommendations[str(userID)] = user_recommendations
        pbar.close()
        return recommendations


The code below calls the package with the SlopeOne algorithm.

In [66]:
modelSlopeOne = SurpriseRecMethod(SlopeOne())
modelSlopeOne.fit(train_set)
recSlopeOne = modelSlopeOne.get_top_n_recommendations(test_set, 5)
p3 = compute_precision(test_set,recSlopeOne)








  0%|          | 0/227 [00:00<?, ?it/s]






  3%|▎         | 6/227 [00:00<00:04, 54.23it/s]






  4%|▍         | 10/227 [00:00<00:04, 48.80it/s]






  8%|▊         | 18/227 [00:00<00:03, 54.00it/s]






 10%|█         | 23/227 [00:00<00:04, 46.23it/s]






 12%|█▏        | 28/227 [00:00<00:04, 43.05it/s]






 15%|█▍        | 34/227 [00:00<00:04, 46.02it/s]






 18%|█▊        | 40/227 [00:00<00:03, 49.31it/s]






 21%|██        | 47/227 [00:00<00:03, 52.90it/s]






 23%|██▎       | 53/227 [00:01<00:03, 52.47it/s]






 26%|██▌       | 59/227 [00:01<00:03, 48.12it/s]






 28%|██▊       | 64/227 [00:01<00:03, 45.32it/s]






 30%|███       | 69/227 [00:01<00:03, 42.88it/s]






 33%|███▎      | 75/227 [00:01<00:03, 45.76it/s]






 35%|███▌      | 80/227 [00:01<00:03, 46.17it/s]






 37%|███▋      | 85/227 [00:01<00:03, 46.77it/s]






 41%|████      | 92/227 [00:01<00:02, 50.26it/s]






 44%|████▎     | 99/227 [00:01<00:02, 54.01it/s]






 46%|████▋   

The code below calls the package with a nearest neighbor user-item recommendation method.

In [67]:
modelKNNUser = SurpriseRecMethod(KNNBasic(sim_options={'name': 'cosine', 'user_based': True}))
modelKNNUser.fit(train_set)
recKNNUser = modelKNNUser.get_top_n_recommendations(test_set, 5)
p4 = compute_precision(test_set,recKNNUser)

Computing the cosine similarity matrix...
Done computing similarity matrix.









  0%|          | 0/227 [00:00<?, ?it/s]






  1%|▏         | 3/227 [00:00<00:08, 25.17it/s]






  2%|▏         | 4/227 [00:00<00:25,  8.71it/s]






  3%|▎         | 6/227 [00:00<00:21, 10.19it/s]






  4%|▎         | 8/227 [00:00<00:18, 11.76it/s]






  4%|▍         | 10/227 [00:00<00:16, 13.10it/s]






  5%|▌         | 12/227 [00:00<00:15, 14.11it/s]






  6%|▌         | 14/227 [00:00<00:14, 14.50it/s]






  7%|▋         | 16/227 [00:01<00:14, 14.72it/s]






  8%|▊         | 18/227 [00:01<00:13, 15.43it/s]






  9%|▉         | 20/227 [00:01<00:13, 15.86it/s]






 10%|▉         | 22/227 [00:01<00:12, 16.84it/s]






 11%|█         | 24/227 [00:01<00:11, 17.30it/s]






 12%|█▏        | 27/227 [00:01<00:11, 17.94it/s]






 13%|█▎        | 29/227 [00:01<00:10, 18.39it/s]






 14%|█▎        | 31/227 [00:01<00:11, 17.73it/s]






 15%|█▍        | 33/227 [00:02<00:11, 17.49it/s]






 15%|█▌        | 35/227 [00:02<00:10, 17.57it/s]






 16%|█▋        |

 99%|█████████▊| 224/227 [00:12<00:00, 17.35it/s]






100%|██████████| 227/227 [00:13<00:00, 17.44it/s]


Let us look at the results of all algorithms together:

In [68]:
pd.DataFrame.from_dict({'Jaccard':p1,'Popularity':p2,'SlopeOne':p3,'User KNN':p4}, orient='index',columns=['Precision'])

,Precision
Jaccard,0.032599
Popularity,0.027313
SlopeOne,0.032599
User KNN,0.075771


Try the NMF (non-negative matrix factorization) algorithms from the package - https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.NMF.


In [69]:
#Your code here
modelNMFUser = SurpriseRecMethod(NMF())
modelNMFUser.fit(train_set)
recNMFUser = modelNMFUser.get_top_n_recommendations(test_set, 5)
p5 = compute_precision(test_set,recNMFUser)








  0%|          | 0/227 [00:00<?, ?it/s]






  4%|▎         | 8/227 [00:00<00:03, 72.42it/s]






  7%|▋         | 16/227 [00:00<00:02, 73.91it/s]






 10%|█         | 23/227 [00:00<00:02, 72.10it/s]






 14%|█▎        | 31/227 [00:00<00:02, 71.84it/s]






 18%|█▊        | 40/227 [00:00<00:02, 74.69it/s]






 21%|██        | 48/227 [00:00<00:02, 75.11it/s]






 25%|██▌       | 57/227 [00:00<00:02, 76.04it/s]






 29%|██▊       | 65/227 [00:00<00:02, 75.36it/s]






 32%|███▏      | 73/227 [00:00<00:02, 75.10it/s]






 36%|███▌      | 81/227 [00:01<00:01, 76.15it/s]






 39%|███▉      | 89/227 [00:01<00:01, 76.64it/s]






 43%|████▎     | 97/227 [00:01<00:01, 77.08it/s]






 47%|████▋     | 106/227 [00:01<00:01, 79.37it/s]






 50%|█████     | 114/227 [00:01<00:01, 77.99it/s]






 54%|█████▍    | 123/227 [00:01<00:01, 79.65it/s]






 58%|█████▊    | 131/227 [00:01<00:01, 79.49it/s]






 62%|██████▏   | 140/227 [00:01<00:01, 80.87it/s]






 66%|███

For this particular dataset, the user nearest neighbor approach worked best. Hence, should we need to choose a method to put online, we should go with this method.

In [70]:
print(p5)

0.03788546255506608
